# Host Conection, Host Group Creation and Volume Provisioning with Active Cluster

![alt text](images/diagram.png "Diagram")

The goal of this notebook is to document the Python scripts used to add new hosts into a Flash Array that is used with Active Cluster. This process involves the mapping of each host to each Flash Array and the creation of a new Host Group that will contain the new cluster.

This is used when new clusters are added into Flash Array and with this script the full process is automated. This include tasks as setting the personality for the hosts.

At the end, we will define the volumes that want to be created and that will also be automatically mapped to the host group.

As seen above, we have two Flash Arrays and two ESX hosts to be mapped.

Also here you have some documentation about Python module:

* __[Documentation Overview](http://pure-storage-python-rest-client.readthedocs.io/en/latest/)__
* __[Installation Instructions](http://pure-storage-python-rest-client.readthedocs.io/en/latest/installation.html)__
* __[purestorage python API glossary](http://pure-storage-python-rest-client.readthedocs.io/en/latest/api.html)__
* __[rest-client](https://github.com/purestorage/rest-client/blob/master/purestorage/purestorage.py)__



## Flash Array Definitions

This is the part you will have to change adn adapt to your environment. So here you will be using your IP addresses, your array names, and your tokens. 

In the Python scripts, this information is included in the definitions_yourarrays.py file

In [13]:
ArrayName1="lon-m20-a"
IP1='x.x.x.x'
token1='1234eafb-4a5d-c1da-d82c-0208b24ba8db'

ArrayName2="lon-m20-b"
IP2='x.x.x.x'
token2='123439b4-572b-b627-9ebf-ac2adef5334d'

![alt text](images/arrays.png "Diagram")

Next we have to define the hosts we want to add to the Flash Array Active Cluster. Here we include the host names, their WWN and their personality. 

This is included in the CLUSTER_yourcluster.py file in the python scripts examples.

You just need to modify this with your own names, WWNs and personalities (if not ESX servers)

In [14]:
HostAttributes = {}
HostAttributes["personality"] = 'esxi'

HostGroupName = 'CLUSTERCC'

#Define here the host names that belong to the host group
HostNames = {}
HostNames["hostlist"] = ['CChost5a','CChost5b']

#Now insert the wwwn for each host
WWN = {"CChost5a" : {"wwnlist" : ["91:42:33:44:55:66:77:85", "92:43:44:55:66:77:88:95"]},
       "CChost5b" : {"wwnlist" : ["9A:42:33:44:55:66:77:85", "9B:43:44:55:66:77:88:95"]}
      }

#Define where each host is located. Their prefered array will be the one in the same data center
PreferedArrays = { "CChost5a" : {"prefered array" : "lon-m20-a"},
                   "CChost5b" : {"prefered array" : "lon-m20-b"}
      }


![alt text](images/esx.png "HOSTS")

Once we have defined our environment, we are going to go step by step describing what we are doing

## Import Libraries

Get the libraries we are going to need

In [15]:
import purestorage

import urllib3
# We're using a version that warns us about insecure requests, and we don't want to see that noise.
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


## Connect to the Flash Arrays

We are going to connect to the FAs and make sure that the IP match with the host name that is provided

In [16]:
try:
    array1 = purestorage.FlashArray(IP1, api_token=token1)

except: 
    print ("Issue connecting with Array: % " % IP)
    raise

    
info1 = array1.get() 
if (info1["array_name"] != ArrayName1):
    print "There is some mis-configuration because array name does not match"
    raise
    
try:
    array2 = purestorage.FlashArray(IP2, api_token=token2)

except: 
    print ("Issue connecting with Array: % " % IP)
    raise
    
    
info2 = array2.get() 
if (info2["array_name"] != ArrayName2):
    print "There is some mis-configuration because array name does not match"
    raise


# Create new Hosts en each of the Flash Arrays

Using the hosts we have previously defined, we are going to use each Flash Array connection to create a new host and set their Properties

In [17]:
if HostNames is not None:   # we have a list of hosts to be connected
    for host in HostNames["hostlist"]:
        wwnlist_host = WWN[host]
                
        try:
            h_info = array1.create_host(host, **wwnlist_host)
            print ("Adding Host: %s to Array: %s" % (host, ArrayName1))
            h_info = array1.set_host(host, **HostAttributes) 
            print ("Setting Attributre: %s on Host: %s and Array: %s" % (HostAttributes["personality"], host, ArrayName1))            
                                     
        except:
            print ("Issue adding host %s at array %s" % (host, ArrayName1))
            raise
     
        try:
            h_info = array2.create_host(host, **wwnlist_host)
            print ("Adding Host: %s to Array: %s" % (host, ArrayName2))
            h_info = array2.set_host(host, **HostAttributes) 
            print ("Setting Attributre: %s on Host: %s and Array: %s" % (HostAttributes["personality"], host, ArrayName2))            

        except:
            print ("Issue adding host %s at array %s" % (host, ArrayName2))
            raise


Adding Host: CChost5a to Array: lon-m20-a
Setting Attributre: esxi on Host: CChost5a and Array: lon-m20-a
Adding Host: CChost5a to Array: lon-m20-b
Setting Attributre: esxi on Host: CChost5a and Array: lon-m20-b
Adding Host: CChost5b to Array: lon-m20-a
Setting Attributre: esxi on Host: CChost5b and Array: lon-m20-a
Adding Host: CChost5b to Array: lon-m20-b
Setting Attributre: esxi on Host: CChost5b and Array: lon-m20-b


## Create the Host Group

Now we add the hosts into the host group we have defined

In [18]:
try:
    hgroup = array1.create_hgroup(HostGroupName, **HostNames)
    print ("Host Group: %s created in Array: %s" % (HostGroupName, ArrayName1))
except:
    print ("Issue creating Host Group %s in Array %s" % (HostGroupName, ArrayName1))
    raise
 
try:
    hgroup = array2.create_hgroup(HostGroupName, **HostNames)
    print ("Host Group: %s created in Array: %s" % (HostGroupName, ArrayName2))
except:
    print ("Issue creating Host Group %s in Array %s" % (HostGroupName, ArrayName2))
    raise


Host Group: CLUSTERCC created in Array: lon-m20-a
Host Group: CLUSTERCC created in Array: lon-m20-b


## Storage Provisioning

This section correspond to the script create_volume_and_map_to_hostg.py script. We are going to split it into pieces in this notebook

First, we are going to define again the  variables that we may need for this provisioning

If you have not done it already, execue the cell with the  Array Definition and the Cell to connect to the Flash Arrays

In [19]:
HostGroupName = 'CLUSTERCC'

threshold = 80

Pod = 'jspod'

LUNS_to_provision=['30G','45G']

## Check Capacity

Let´s make sure that each array do not pass the threshold defined. You can modify that value yourself and even avoid this section

In [20]:
array_info1 = array1.get(space="true")
for info in array_info1:
    capacity = info["capacity"]/1024/1024/1024
    total = (info["total"]/1024/1024/1024)
    utilization = float(100*total/capacity)
    print ("Array: %s" % (ArrayName1))
    print ("Total Capacity: %d GiB, Total Used: %d GiB" % (capacity, total))
    print ("Percentage capacity used: %d percent" % (utilization))

    
if utilization > threshold:
        print ("Space utilization is %d, make sure you have enough space before creating volumes" % utilization)
        print ("Volume not created")
        exit()
          

array_info2 = array2.get(space="true")
for info in array_info2:
    capacity = info["capacity"]/1024/1024/1024
    total = (info["total"]/1024/1024/1024)
    utilization = float(100*total/capacity)
    print ("Array: %s" % (ArrayName2))
    print ("Total Capacity: %d GiB, Total Used: %d GiB" % (capacity, total))
    print ("Percentage capacity used: %d percent" % (utilization))

if utilization > threshold:
        print ("Space utilization is %d, make sure you have enough space before creating volumes" % utilization)
        print ("Volume not created")
        exit()
          

Array: lon-m20-a
Total Capacity: 2858 GiB, Total Used: 503 GiB
Percentage capacity used: 17 percent
Array: lon-m20-b
Total Capacity: 2858 GiB, Total Used: 482 GiB
Percentage capacity used: 16 percent


## Create Volumes

The voume name is going to be given based on the Host Group Name and in the last 4 numbers for the LUN ID. Therfore, we first create a volume using a temporry name, then get their ID, compose the new name and rename it.

We store the name of the LUNs created into a list so we can map it later to the host group

In [21]:
LUNS_created = []

TempName = Pod + "::" + "Temporal"

for LUN in LUNS_to_provision:
    try:
        newvol = array1.create_volume(TempName, LUN)
    except:
        print "Error while creating the volume"
        raise    
    
    serial = newvol["serial"]
    newname = Pod + "::" + HostGroupName + "-" + serial[-4:]
    
    rename = array1.rename_volume(TempName, newname)
    
    print ("Created new volume: {} of size {}").format(newname, LUN)

    LUNS_created.append(newname)        

Created new volume: jspod::CLUSTERCC-1374 of size 30G
Created new volume: jspod::CLUSTERCC-1375 of size 45G


## Connect the new volumes to the host group

Now we take the list of the volumes created and we map it to the Host Group

In [22]:

for volume in LUNS_created:

    try:
        array1.connect_hgroup(HostGroupName, volume)
        array2.connect_hgroup(HostGroupName, volume)
        print ("Volume: %s, conected to Host Group: %s from both arrays" % (volume, HostGroupName))
    except:
        print ("There was some issue connecting Host Group: %s, with volume %s. Please check" % (HostGroupName, volume))
        raise


Volume: jspod::CLUSTERCC-1374, conected to Host Group: CLUSTERCC from both arrays
Volume: jspod::CLUSTERCC-1375, conected to Host Group: CLUSTERCC from both arrays


## Verification

We can take a look to eachof the Flash Arrays and make sure that the configurion has worked as expected

Here we can see how the two volumes have been created:

![alt text](images/fa1.png "HOSTS")

How a volume is mapped to the Host Group

![alt text](images/fa2.png "HOSTS")

And how the Host Group has been created with the proper members and the two volumes have been mapped properly

![alt text](images/fa3.png "HOSTS")